In [1]:
import sys,os
#sys.path.append("/home/wardlow/denis/mypy")
import xuleta as xu
from scipy.stats import *
import pylab as plt
from scipy import interpolate
from glob import glob
import numpy as np
%matplotlib inline


In [2]:
shape = "/media/denis/seagate/PB/wet_seasons.shp"
fout = "/media/denis/seagate/PBv2/outputs/"

#list
wets = ["'FMAM'","'AMJJ'","'DJFM'","'JFMA'","'NDJF'"]
season = [['02','03','04','05'],
         ['04','05','06','07'],
         ['12','01','02','03'],
         ['01','02','03','04'],
         ['11','12','01','02']]

In [5]:
def mk_images(folder,name,full=True):
    for wet,months in zip(wets,season):
        
        if full==True:
            lista = [x for x in sorted(glob(folder+"/*.tif")) if x[-6:-4] in months]
        else:
            lista = [x for x in sorted(glob(folder+"/*.tif")) if x[-6:-4] in months and x[-11:-7] in [str(y) for y in range(2002,2012)]]
        print(len(lista))

        #creating mask
        os.system('gdalwarp -cutline %s -cwhere "WET = %s" -co COMPRESS=DEFLATE -co PREDICTOR=1 -co ZLEVEL=9 -dstnodata -9999 %s %smask%s.tif' %(shape,wet,lista[0],fout,wet[1:-1]))
        mask,mm = xu.TifToArray(fout+'mask'+wet[1:-1]+".tif")
        mask = np.where(mask!=-9999,1,0)
        print('Mask %s created' %wet)

        # reading files
        arrays = []
        for i in lista:
            a,m = xu.TifToArray(i)
            arrays.append(a)
        arrays3 = np.asarray(arrays)
        arrays3 = np.reshape(arrays3,(arrays3.shape[0],arrays3.shape[1]*arrays3.shape[2]))
        maskrs = np.reshape(mask,(1,arrays3.shape[1]))
        print(arrays3.shape)

        #Running masked Theil-Sen
        x = np.arange(len(lista))

        # creating matrices for outputs
        zslope = np.zeros(arrays3.shape[1])
        pvalue = np.zeros(arrays3.shape[1])
        
        print('Running Mann-Kendall! Go home!')
        for i in range(arrays3.shape[1]): #range(1,2000000,1)range(arrays3.shape[1])
            #apply mask
            if maskrs[:,i] == 1:
                ts_results = xu.mk_test(interpolate(arrays3[:,i]),alpha=0.05)
                zslope[i] = ts_results[2]
                pvalue[i] = ts_results[1]
                #lo_slope[i] = ts_results[2]
                #up_slope[i] = ts_results[3]
                ts_results = None
            else:
                zslope[i] = np.nan
                pvalue[i] = np.nan
               
        zslope = zslope.reshape(a.shape[0],a.shape[1])
        pvalue = pvalue.reshape(a.shape[0],a.shape[1])
       
        #SAVE RESULTS
        xu.ArrayToTif(zslope,name+"_zslope"+wet[1:-1]+".tif",fout+'mann-kendall/',m,Type=3)
        xu.ArrayToTif(pvalue,name+"_pvalue"+wet[1:-1]+".tif",fout+'mann-kendall/',m,Type=3)

        print('%s saved' %wet)

        #os.remove(fout+'mask'+wet[1:-1]+".tif")
        if os.path.exists(fout+'mask.tif'): os.remove(fout+'mask.tif')

In [6]:
name = 'lai_full'
folder = "/media/denis/seagate/PBv2/mcd15a2h/raw/filt_2/anomalies/monthly_mean"
mk_images(folder=folder,name=name,full=False)

36
Mask 'FMAM' created
(36, 13984056)
Running Mann-Kendall! Go home!


TypeError: 'module' object is not callable